In [18]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import msoffcrypto
from io import BytesIO
from openpyxl import load_workbook, Workbook

# 컬럼 매핑 규칙
COLUMN_MAPPING = {
    "수취고객명": "수취인명",
    "수취인": "수취인명",
    "수취인전화": "수취인연락처1",
    "수취인휴대폰": "수취인연락처1", 
    "수취인우편번호": "우편번호",
    "수취인주소": "통합배송지",
    "총중량": "수량",
    "상품명1": "상품명",
    "수량1": "수량",
    "물품가격1": "최초 상품별 총 주문금액",
    "MEMO": "옵션정보",
    "총수량": "수량",
    "고객주문번호": "상품주문번호"
}

# 파일 경로 변수
source_file_path = None

# 파일 선택 함수
def select_file():
    global source_file_path
    source_file_path = filedialog.askopenfilename(
        title="원본 엑셀 파일 선택",
        filetypes=[("Excel files", "*.xlsx")]
    )
    if source_file_path:
        lbl_file_path.config(text=f"선택된 파일: {source_file_path}")

# 새로운 엑셀 파일 생성 함수
def generate_file():
    if not source_file_path:
        messagebox.showerror("오류", "먼저 원본 파일을 선택하세요!")
        return

    password = tk.simpledialog.askstring("비밀번호 입력", "원본 파일의 비밀번호를 입력하세요:", show='*')

    if not password:
        messagebox.showerror("오류", "비밀번호를 입력하세요!")
        return

    try:
        decrypted = BytesIO()
        with open(source_file_path, "rb") as f:
            file = msoffcrypto.OfficeFile(f)
            file.load_key(password=password)
            file.decrypt(decrypted)

        decrypted.seek(0)
        wb = load_workbook(decrypted)
        ws = wb.active
        data = list(ws.values)
        columns = data[1]  # 첫 번째 행은 헤더
        source_data = pd.DataFrame(data[2:], columns=columns)  # 2행부터 데이터 시작

        # 새로운 데이터프레임 생성
        new_data = pd.DataFrame()
        for target_col, source_col in COLUMN_MAPPING.items():
            try:
                # 데이터 타입 변환 및 결측치 처리 (필요한 경우)
                new_data[target_col] = source_data[source_col].astype(str).fillna('')
            except KeyError:
                print(f"컬럼 '{source_col}'이 존재하지 않습니다.")
                new_data[target_col] = ""

        # 데이터프레임 확인 (디버깅용)
        # print(new_data.head())

        # 파일 저장
        save_path = filedialog.asksaveasfilename(
            title="저장할 파일 선택",
            defaultextension=".xlsx",
            filetypes=[("Excel files", "*.xlsx")]
        )
        if save_path:
            new_wb = Workbook()
            new_ws = new_wb.active
            new_ws.append(new_data.columns.tolist())
            for _, row in new_data.iterrows():
                new_ws.append(row.tolist())
            new_wb.save(save_path)
            messagebox.showinfo("성공", f"새로운 엑셀 파일이 저장되었습니다:\n{save_path}")
    except Exception as e:
        messagebox.showerror("오류", f"오류 발생: {str(e)}")

# GUI 생성
root = tk.Tk()
root.title("엑셀 데이터 변환")

# 파일 찾기 버튼
btn_select_file = tk.Button(root, text="파일 찾기", command=select_file)
btn_select_file.pack(pady=10)

# 선택된 파일 경로 표시
lbl_file_path = tk.Label(root, text="선택된 파일: 없음", wraplength=400)
lbl_file_path.pack(pady=10)

# 파일 생성 버튼
btn_generate_file = tk.Button(root, text="파일 변환", command=generate_file)
btn_generate_file.pack(pady=10)

root.mainloop()

  수취고객명  수취인           수취인전화          수취인휴대폰 수취인우편번호  \
0    허웅   허웅   010-3747-3412   010-3747-3412   06627   
1   현진엽  현진엽  0502-3165-5629  0502-3165-5629   03184   

                                        수취인주소 총중량  \
0  서울특별시 서초구 강남대로51길 10 (서초동, 강남효성해링턴타워) 818호   1   
1  서울특별시 종로구 새문안로2길 10 (신문로2가) 디팰리스 아파트 1301호   1   

                                                상품명1 수량1  물품가격1  \
0  미스터불스 쏘팔메토 옥타코사놀 아연 비타민 프리미엄 남성 활력영양제 1,362mg ...   1  68000   
1  미스터불스 쏘팔메토 옥타코사놀 아연 비타민 프리미엄 남성 활력영양제 1,362mg ...   1  67000   

               MEMO 총수량            고객주문번호  
0  수량: 1병 / 선택: 60정   1  2025011950833441  
1  수량: 1병 / 선택: 60정   1  2025011945367251  
  수취고객명  수취인           수취인전화          수취인휴대폰 수취인우편번호  \
0    허웅   허웅   010-3747-3412   010-3747-3412   06627   
1   현진엽  현진엽  0502-3165-5629  0502-3165-5629   03184   

                                        수취인주소 총중량  \
0  서울특별시 서초구 강남대로51길 10 (서초동, 강남효성해링턴타워) 818호   1   
1  서울특별시 종로구 새문안로2길 10 (신문로2가) 디팰리스 아파트 1301호   1   

        